In [1]:
from fbprophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [2]:
model = Prophet(changepoint_prior_scale=0.01)

In [3]:
import pandas as pd

In [16]:
df = pd.read_excel('Actuals.xlsx')

In [17]:
df.head()

,Time,Load (kW),Pressure_kpa,Cloud Cover (%),Humidity (%),Temperature (C),Wind Direction (deg),Wind Speed (kmh)
0,2017-03-18 00:00:00,1.031472e+06,1012.48000,2.125,79.333333,16.764386,285.25,6.25375
1,2017-03-18 01:00:00,1.007206e+06,1011.61875,0.000,82.166667,15.729386,285.00,9.20500
2,2017-03-18 02:00:00,9.861084e+05,1010.72000,2.125,81.666667,15.816886,316.00,11.28375
3,2017-03-18 03:00:00,9.707610e+05,1011.37625,17.125,82.250000,15.919386,327.00,7.16875
4,2017-03-18 04:00:00,9.622584e+05,1011.50000,18.625,82.333333,15.769386,282.25,0.78000


In [19]:
dfu = df[['Time', 'Humidity (%)', 'Temperature (C) ', 'Load (kW)']]

In [20]:
dfu

,Time,Humidity (%),Temperature (C),Load (kW)
0,2017-03-18 00:00:00,79.333333,16.764386,1.031472e+06
1,2017-03-18 01:00:00,82.166667,15.729386,1.007206e+06
2,2017-03-18 02:00:00,81.666667,15.816886,9.861084e+05
3,2017-03-18 03:00:00,82.250000,15.919386,9.707610e+05
4,2017-03-18 04:00:00,82.333333,15.769386,9.622584e+05
...,...,...,...,...
31907,2020-11-06 11:00:00,68.166667,21.299386,1.242830e+06
31908,2020-11-06 12:00:00,67.166667,22.419386,1.239480e+06
31909,2020-11-06 13:00:00,67.583333,22.971886,1.234773e+06
31910,2020-11-06 14:00:00,66.250000,24.314386,1.240140e+06


In [44]:
dfother = pd.read_csv('model_ready.csv')

In [45]:
def dataPreprocessing(dataFrame):
    dataFrame['Date'] = pd.to_datetime(dataFrame['Date']) + dataFrame['hour'].astype('timedelta64[h]') + dataFrame['hour_part'].astype('timedelta64[m]')
    dataFrame.drop(columns=['hour'], axis=1, inplace=True)
    dataFrame.drop(columns=['hour_part'], axis=1, inplace=True)
    return dataFrame

In [46]:
dfother

,Haryana Load (MW),month,hour,weekday,temp,dew,humidity,windspeed,winddir,visibility,cloudcover,hour_part,year,day,Date
0,7224.17,10,0,5,30.0,24.0,70.29,0.0,0.0,2.0,27.4,15,2016,1,2016-10-01
1,7141.92,10,0,5,30.0,24.0,70.29,0.0,0.0,2.0,27.4,30,2016,1,2016-10-01
2,7121.93,10,0,5,30.0,24.0,70.29,0.0,0.0,2.0,27.4,45,2016,1,2016-10-01
3,7066.64,10,1,5,30.0,24.0,70.29,0.0,0.0,2.0,0.0,0,2016,1,2016-10-01
4,6937.04,10,1,5,30.0,24.0,70.29,0.0,0.0,2.0,0.0,15,2016,1,2016-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224632,8537.11,9,23,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,0,2021,9,2021-09-09
224633,8554.19,9,23,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,15,2021,9,2021-09-09
224634,8536.29,9,23,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,30,2021,9,2021-09-09
224635,8510.32,9,23,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,45,2021,9,2021-09-09


In [47]:
dataotherpp = dataPreprocessing(dfother)

In [48]:
dataotherpp

,Haryana Load (MW),month,weekday,temp,dew,humidity,windspeed,winddir,visibility,cloudcover,year,day,Date
0,7224.17,10,5,30.0,24.0,70.29,0.0,0.0,2.0,27.4,2016,1,2016-10-01 00:15:00
1,7141.92,10,5,30.0,24.0,70.29,0.0,0.0,2.0,27.4,2016,1,2016-10-01 00:30:00
2,7121.93,10,5,30.0,24.0,70.29,0.0,0.0,2.0,27.4,2016,1,2016-10-01 00:45:00
3,7066.64,10,5,30.0,24.0,70.29,0.0,0.0,2.0,0.0,2016,1,2016-10-01 01:00:00
4,6937.04,10,5,30.0,24.0,70.29,0.0,0.0,2.0,0.0,2016,1,2016-10-01 01:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224632,8537.11,9,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,2021,9,2021-09-09 23:00:00
224633,8554.19,9,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,2021,9,2021-09-09 23:15:00
224634,8536.29,9,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,2021,9,2021-09-09 23:30:00
224635,8510.32,9,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,2021,9,2021-09-09 23:45:00


In [50]:
dataotherpp.to_csv('model_ready_ts.csv')

In [53]:
dftest = pd.read_csv('model_ready_ts.csv')

In [54]:
dftest

,Unnamed: 0,Haryana Load (MW),month,weekday,temp,dew,humidity,windspeed,winddir,visibility,cloudcover,year,day,Date
0,0,7224.17,10,5,30.0,24.0,70.29,0.0,0.0,2.0,27.4,2016,1,2016-10-01 00:15:00
1,1,7141.92,10,5,30.0,24.0,70.29,0.0,0.0,2.0,27.4,2016,1,2016-10-01 00:30:00
2,2,7121.93,10,5,30.0,24.0,70.29,0.0,0.0,2.0,27.4,2016,1,2016-10-01 00:45:00
3,3,7066.64,10,5,30.0,24.0,70.29,0.0,0.0,2.0,0.0,2016,1,2016-10-01 01:00:00
4,4,6937.04,10,5,30.0,24.0,70.29,0.0,0.0,2.0,0.0,2016,1,2016-10-01 01:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224632,224632,8537.11,9,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,2021,9,2021-09-09 23:00:00
224633,224633,8554.19,9,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,2021,9,2021-09-09 23:15:00
224634,224634,8536.29,9,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,2021,9,2021-09-09 23:30:00
224635,224635,8510.32,9,3,29.0,26.8,87.64,7.8,82.0,2.6,70.3,2021,9,2021-09-09 23:45:00
